# Pre-procesamiento de los datos

Se importan las librerías a utilizar

In [ ]:
import pandas as pd
import numpy as np 
import glob
from matplotlib import pyplot as plt
from sklearn.cluster import KMeans
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder

Se crea el dataframe

In [ ]:
path = 'base' 
all_files = glob.glob(path + "/*.csv")

li = []

for filename in all_files:
    df = pd.read_csv(filename, index_col=None, sep=';', dtype={'rut': str})
    li.append(df)

big_df = pd.concat(li, axis=0, ignore_index=True)
big_df.head(5)

Se eliminan columnas y valores innecesarios

In [ ]:
big_df = big_df.dropna()
big_df = big_df.reset_index(drop=True)

big_df.drop(big_df[big_df.rut == '25'].index,inplace=True)
big_df.drop('visitasOtros_2018',axis=1,inplace=True)
big_df.drop('visitasOtros_2019',axis=1,inplace=True)
big_df.drop('visitasOtros_2020',axis=1,inplace=True)
big_df.drop('visitasOtros_2021',axis=1,inplace=True)

big_df.head(5)

La columna 'rut' también va a ser eliminada, sin embargo, se va a necesitar luego de usar K-Means, por lo que hay que guardarla antes de eliminarla. 

In [ ]:
df_rut = big_df['rut']
df_rut = df_rut.to_numpy()

big_df.drop('rut',axis=1,inplace=True)
big_df.head(5)

Ahora, hay que codificar las columnas que tienen datos categóricos. Cabe destacar que cada una de las columnas a codificar tiene su propio codificador para poder guardar en él los datos que tiene cada columna, de manera que se pueda revertir el proceso una vez se haya utilizado K-Means.

In [ ]:
le_rangoEdad = LabelEncoder()
le_plan = LabelEncoder()
le_fidelizacion = LabelEncoder()

In [ ]:
rangoEdad_n = le_rangoEdad.fit_transform(big_df['rangoEdad'])
big_df.insert(1, column='rangoEdad_n', value=rangoEdad_n)
big_df.drop('rangoEdad',axis=1,inplace=True)

plan_n = le_plan.fit_transform(big_df['plan'])
big_df.insert(2, column='plan_n', value=plan_n)
big_df.drop('plan',axis=1,inplace=True)

fidelizacion_n = le_fidelizacion.fit_transform(big_df['fidelizacion'])
big_df.insert(3, column='fidelizacion_n', value=fidelizacion_n)
big_df.drop('fidelizacion',axis=1,inplace=True)

big_df

Se escalan los datos

In [ ]:
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(big_df)

# Algoritmo K-Means

### Análisis del método del codo

In [ ]:
clusters_range = range(1,35)
inertia = []

for k in clusters_range:
    km = KMeans(n_clusters=k)
    km.fit(scaled_data)
    inertia.append(km.inertia_)

In [ ]:
plt.figure(figsize=(15,7))
plt.title('Método del codo')
plt.xlabel('Cantidad de clusters')
plt.ylabel('Inercia (error)')
plt.plot(clusters_range,inertia)
plt.xticks(np.arange(min(clusters_range), max(clusters_range)+1, 1.0))
plt.grid()
plt.show()

### K-Means

In [ ]:
km = KMeans(n_clusters=19)
y_predicted = km.fit_predict(scaled_data)
y_predicted

# Post-procesamiento de datos

Se invierten los datos escalados para poder verlos naturalmente de nuevo y se agrega la columna "cluster", que corresponde a la segmentación creada con K-Means

In [ ]:
big_df['cluster'] = y_predicted

Se decodifican las variables con datos categóricos (rangoEdad, plan y fidelización) para poder entenderlas más fácilmente.

In [ ]:
inverse_rangoEdad = le_rangoEdad.inverse_transform(big_df['rangoEdad_n'])
big_df.insert(1, column='rangoEdad', value=inverse_rangoEdad)
big_df.drop('rangoEdad_n',axis=1,inplace=True)

inverse_plan = le_plan.inverse_transform(big_df['plan_n'])
big_df.insert(2, column='plan', value=inverse_plan)
big_df.drop('plan_n',axis=1,inplace=True)

inverse_fidelizacion = le_fidelizacion.inverse_transform(big_df['fidelizacion_n'])
big_df.insert(3, column='fidelizacion', value=inverse_fidelizacion)
big_df.drop('fidelizacion_n',axis=1,inplace=True)

Finalmente, se agrega nuevamente la columna rut.

In [ ]:
big_df.insert(0, column='rut',value=df_rut)
big_df

In [ ]:
big_df.to_csv('datos_clusterizados.csv', index=False)